In [1]:
import numpy as np
import nltk
import string
import requests
import re
import glob
import os, sys
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
import xlrd
import csv
import time
import dask
import dask.dataframe as dd
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from data_preparation import clean_ticker, download_to_item1
dl = Downloader("/home/hongzhuoqiao/10K_Projects/sec_filings")

In [2]:
dl.get("10-K", 'A')

21

In [4]:
ticker = 'A'
path_header = "/home/hongzhuoqiao/10K_Projects/sec_filings/sec_edgar_filings"
path_for_extraction_item1 = path_header + "/" + ticker + "/10-K" +"/*.txt"
path_for_item1_folder = "/home/hongzhuoqiao/10K_Projects/sec_filings/item1_section/" + ticker

# Test code for extract single company's file
file_name = "0000912057-00-002160.txt"
single_file_path = path_header + "/" + ticker + "/10-K/" + file_name

with open(single_file_path, "r") as f:
    raw_10k = f.read()
print(raw_10k[0:1000])

-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: webmaster@www.sec.gov
Originator-Key-Asymmetric:
 MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen
 TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB
MIC-Info: RSA-MD5,RSA,
 U7zCYL+YukoJ5nMfZFUPdtROw5Q9seOi7L4FtynMQyCM1QNrOYdZx7lFKVGIr4/E
 BZPgpB1UfFCPv+J4w36J/Q==

<SEC-DOCUMENT>0000912057-00-002160.txt : 20000203
<SEC-HEADER>0000912057-00-002160.hdr.sgml : 20000203
ACCESSION NUMBER:		0000912057-00-002160
CONFORMED SUBMISSION TYPE:	10-K405
PUBLIC DOCUMENT COUNT:		5
CONFORMED PERIOD OF REPORT:	19991031
FILED AS OF DATE:		20000125

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			AGILENT TECHNOLOGIES INC
		CENTRAL INDEX KEY:			0001090872
		STANDARD INDUSTRIAL CLASSIFICATION:	INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY & ELEC SIGNALS [3825]
		IRS NUMBER:				770518772
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1031

	FILING VALUES:
		FORM TYPE:		10-K405
		SEC ACT:		
		SE

In [5]:
## extraction cell for 1st initial batch extraction. always syncrynized with the "mult_company_item1_extration.ipynb"
# Regex to find <DOCUMENT> tags
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')

### as section names
doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

#regex = re.compile(r'(Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0,1})|(>Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0,1})|(ITEM(\s|&#160;|&nbsp;)(1A|1B|1|2).{0,1})')

regex = re.compile(r'(Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0})|(>Item(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0})|(ITEM(\s|&#160;|&nbsp;)(1A|1B|1|2)\.{0})')

# Create a loop to go through each section type and save only the 10-K section in the dictionary
document = {}
doc = ''
extract_failed_flag = 0

for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    
    if (doc_type =='10-K'):
        document[doc_type] = raw_10k[doc_start:doc_end]    
        # Use finditer to math the regex
        matches = regex.finditer(document['10-K'])
        doc = '10-K'
        
        # Write a for loop to print the matches
        
    elif (doc_type =='10-K405'):
        document[doc_type] = raw_10k[doc_start:doc_end]    
        # Use finditer to math the regex
        matches = regex.finditer(document['10-K405'])
        doc = '10-K405'

    elif (doc_type =='10-KT'):
        document[doc_type] = raw_10k[doc_start:doc_end]    
        # Use finditer to math the regex
        matches = regex.finditer(document['10-KT'])
        doc = '10-KT'


# # Create the dataframe
try:
    matches = regex.finditer(document[doc])
    test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

    test_df.columns = ['item', 'start', 'end']
    test_df['item'] = test_df.item.str.lower()

    # Get rid of unnesesary charcters from the dataframe
    test_df.replace('&#160;',' ',regex=True,inplace=True)
    test_df.replace('&nbsp;',' ',regex=True,inplace=True)
    test_df.replace(' ','',regex=True,inplace=True)
    test_df.replace('\.','',regex=True,inplace=True)
    test_df.replace('>','',regex=True,inplace=True)
    test_df.replace('<','',regex=True,inplace=True)
    test_df.replace('\n','',regex=True,inplace=True)
    # shift dataset to find the next, next next item
    test_df['next_1'] = test_df['item'].shift(-1,fill_value=0)
    test_df['next_1_start'] = test_df['start'].shift(-1,fill_value=0)
    test_df['next_1_start'] = test_df.next_1_start.astype('int32')
    # test_df['next_1_end'] = test_df['end'].shift(-1,fill_value=0)
    # test_df['next_1_end'] = test_df.next_1_end.astype('int32')
    test_df['pre_1'] = test_df['item'].shift(1,fill_value=0)
    test_df['pre_1_start'] = test_df['start'].shift(1,fill_value=0)
    test_df['pre_1_start'] = test_df.pre_1_start.astype('int32')

except:
    extract_failed_flag = 1

print (extract_failed_flag)
print (matches)
test_df

0


,item,start,end,next_1,next_1_start,pre_1,pre_1_start
0,item1,4704,4710,item1,6944,0,0
1,item1,6944,6950,item1,10759,item1,4704
2,item1,10759,10765,item1,14756,item1,6944
3,item1,14756,14762,item1,18670,item1,10759
4,item1,18670,18676,item1,22622,item1,14756
5,item1,22622,22628,item1,25953,item1,18670
6,item1,25953,25959,item1,29659,item1,22622
7,item1,29659,29665,item1,33902,item1,25953
8,item1,33902,33908,item1,37685,item1,29659
9,item1,37685,37691,item1,41580,item1,33902


In [6]:
# test cell, exact page number test cell
item_1_raw = document[doc][67623:451250]
item_1_content = BeautifulSoup(item_1_raw, 'lxml')
print(item_1_content.get_text("\n")[0:1000])

and
server-to-server connectivity. The Fibre Channel interconnect protocol, a
standard for the transfer of information between computers and storage devices
defined by the American National Standards Institute, was developed to meet this
growing demand. Fibre Channel supports the transfer of large amounts of data
within storage area networks at speeds of one gigabit per second and greater and
provides high transmission reliability. We supply a broad range of integrated
circuits, fiber optic components and systems to manufacturers of storage area
network systems utilizing the Fibre Channel protocol, such as Data General
Corporation, EMC Corporation, Hitachi and NEC.

    WIRELESS COMMUNICATIONS.  Worldwide subscriber growth for wireless
communications has been increasing rapidly in recent years. The Strategis Group
anticipates that the worldwide market for cellular services will increase to
approximately 650 million subscribers by 2002 from approximately 300 million
subscribers in 1998.

In [156]:
# test cell
df_2 = test_df[((test_df['item'] == 'item1') & (test_df['next_1'] == 'item2'))|((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1'))]
df_2 = df_2[['item', 'start', 'end']]
df_2['next_item'] = df_2['item'].shift(-1,fill_value=0)
df_2['next_item_start'] = df_2['start'].shift(-1,fill_value=0)
df_2['next_item_start'] = df_2.next_item_start.astype('int32')
df_2 = df_2[(df_2['item']=='item1') & (df_2['next_item']=='item2')]
df_2['difference'] = df_2['next_item_start'] - df_2['start']
df_2 = df_2[(df_2['item']=='item1')& (df_2['start'] < 100000)]
df_2.sort_values('difference', ascending = False, inplace = True)
raw = document[doc][df_2.iloc[0]['start']:df_2.iloc[0]['next_item_start']]
item_1 = BeautifulSoup(raw, 'lxml')
print(item_1.get_text("\n")[0:3500])

# df_2.sort_values('difference', ascending = False, inplace = True)
# 


ITEM 1.    BUSINESS

(a)    GENERAL DEVELOPMENT OF BUSINESS
       The Black & Decker Corporation  (collectively with its subsidiaries,  the
       Corporation),  incorporated in Maryland in 1910, is a global marketer and
       manufacturer  of  quality  products  used in and  around the home and for
       commercial applications.  With products and services marketed in over 100
       countries,  the Corporation enjoys worldwide  recognition of strong brand
       names and a superior  reputation  for quality,  design,  innovation,  and
       value.
           The  Corporation  is one of the world's  leading  producers  of power
       tools, power tool accessories, and residential security hardware, and the
       Corporation's  product lines hold leading market share positions in these
       industries.  The  Corporation  is a major global  supplier of  engineered
       fastening and assembly  systems.  The  Corporation  is one of the leading
       producers  of faucets in Nor

In [7]:
## beta cell
try:
    # doc contains 1a 1b section (newr files)
    if len(test_df[test_df['item'].str.contains('item1a')]):
        df = test_df[((test_df['item'] == 'item1') & (test_df['next_1'] == 'item1a'))|((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1b')) | ((test_df['item'] == 'item1') & (test_df['next_1'] == 'item2')) | ((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1a'))] 
        df = df[['item', 'start', 'end']]
        df['next_item'] = df['item'].shift(-1,fill_value=0)
        df['next_item_start'] = df['start'].shift(-1,fill_value=0)
        df['next_item_start'] = df.next_item_start.astype('int32')
        df = df[(df['item']=='item1') & (df['next_item']=='item2')]
        df['difference'] = df['next_item_start'] - df['start']
        df.sort_values('difference', ascending = False, inplace = True)
        item_1_raw = document[doc][df.iloc[0]['start']:df.iloc[0]['next_item_start']]


    # doc no 1a 1b
    else:
        df_1 = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='first')
        df_1.set_index('item', inplace=True)
        raw_1 = document[doc][df_1['start'].loc['item1']:df_1['start'].loc['item2']]
        
        df_2 = test_df[((test_df['item'] == 'item1') & (test_df['next_1'] == 'item2'))|((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1'))]
        df_2 = df_2[['item', 'start', 'end']]
        df_2['next_item'] = df_2['item'].shift(-1,fill_value=0)
        df_2['next_item_start'] = df_2['start'].shift(-1,fill_value=0)
        df_2['next_item_start'] = df_2.next_item_start.astype('int32')
        df_2 = df_2[(df_2['item']=='item1') & (df_2['next_item']=='item2')]
        df_2['difference'] = df_2['next_item_start'] - df_2['start']
        df_2 = df_2[(df_2['item']=='item1')& (df_2['start'] < 100000)]
        df_2.sort_values('difference', ascending = False, inplace = True)
        raw_2 = document[doc][df_2.iloc[0]['start']:df_2.iloc[0]['next_item_start']]

        if (len(raw_1)>=len(raw_2)):
            item_1_raw = raw_1
        else:
            item_1_raw = raw_2

    item_1_content = BeautifulSoup(item_1_raw, 'lxml')
    print(item_1_content.get_text("\n")[0:3500])

except:
    print ("OPOOS!...Something Wrong!")

OPOOS!...Something Wrong!


In [135]:
## original finised cell
try:
    # doc contains 1a 1b section (newr files)
    if len(test_df[test_df['item'].str.contains('item1a')]):
        df = test_df[((test_df['item'] == 'item1') & (test_df['next_1'] == 'item1a'))|((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1b')) | ((test_df['item'] == 'item1') & (test_df['next_1'] == 'item2')) | ((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1a'))] 
        df = df[['item', 'start', 'end']]
        df['next_item'] = df['item'].shift(-1,fill_value=0)
        df['next_item_start'] = df['start'].shift(-1,fill_value=0)
        df['next_item_start'] = df.next_item_start.astype('int32')
        df = df[(df['item']=='item1') & (df['next_item']=='item2')]
        df['difference'] = df['next_item_start'] - df['start']
        df.sort_values('difference', ascending = False, inplace = True)
        item_1_raw = document[doc][df.iloc[0]['start']:df.iloc[0]['next_item_start']]


    # doc no 1a 1b
    else:
        df_1 = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='first')
        df_1.set_index('item', inplace=True)
        raw_1 = document[doc][df_1['start'].loc['item1']:df_1['start'].loc['item2']]
        df_2 = test_df[(test_df['item'] == 'item1') & (test_df['next_1'] == 'item2')]
        df_2['difference'] = test_df['next_1_start'] - test_df['start']
        df_2.sort_values('difference', ascending = False, inplace = True)
        raw_2 = document[doc][df_2.iloc[0]['start']:df_2.iloc[0]['next_1_start']]

        if (len(raw_1)>=len(raw_2)):
            item_1_raw = raw_1
        else:
            item_1_raw = raw_2

    item_1_content = BeautifulSoup(item_1_raw, 'lxml')
    print(item_1_content.get_text("\n")[0:3500])

except:
    print ("OPOOS!...Something Wrong!")


ITEM 14.  EXHIBITS, FINANCIAL STATEMENT SCHEDULES AND                    
          REPORTS ON FORM 8-K

(a)  List of Financial Statements, Financial Statement Schedules, and Exhibits

     (1)  List of Financial Statements
          The following consolidated financial statements of the Corporation and
          its subsidiaries are included in Item 8 of Part II:

               Consolidated  Statement  of Earnings - years ended  December  31,
               1998, 1997, and 1996.

               Consolidated Balance Sheet - December 31, 1998 and 1997.

               Consolidated  Statement  of  Stockholders'  Equity - years  ended
               December 31, 1998, 1997, and 1996.

               Consolidated  Statement of Cash Flows - years ended  December 31,
               1998, 1997, and 1996.

               Notes to Consolidated Financial Statements.

               Report of Independent Auditors.


     (2)  List of Financial Statement Schedules
          The following financia

In [11]:
# docs contains 1a 1b
df = test_df[((test_df['item'] == 'item1') & (test_df['next_1'] == 'item1a'))|((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1b')) | ((test_df['item'] == 'item1') & (test_df['next_1'] == 'item2')) | ((test_df['item'] == 'item2') & (test_df['pre_1'] == 'item1a'))] 
df = df[['item', 'start', 'end']]
df['next_item'] = df['item'].shift(-1,fill_value=0)
df['next_item_start'] = df['start'].shift(-1,fill_value=0)
df['next_item_start'] = df.next_item_start.astype('int32')
df = df[(df['item']=='item1') & (df['next_item']=='item2')]
df['difference'] = df['next_item_start'] - df['start']
df.sort_values('difference', ascending = False, inplace = True)
item_1_raw = document[doc][df.iloc[0]['start']:df.iloc[0]['next_item_start']]
item_1_content = BeautifulSoup(item_1_raw, 'lxml')
print(item_1_content.get_text("\n")[0:3500])
# df

>Item 1. Business




General




     IKONICS Corporation (IKONICS or the Company) was incorporated in Minnesota as Chroma-Glo,
Inc. in 1952 and changed its name to The Chromaline Corporation in 1982. In December 2002, the
Company changed its name to IKONICS Corporation. The Company develops, manufactures and sells
light-sensitive liquid coatings (emulsions) and films for screen printing and abrasive etching.
The Company also markets inkjet receptive films and ancillary chemicals. The Company resells
equipment and other consumables to provide a full line of products and services to its customers.
In 2006, the Company began to offer custom etching services for silicon wafers, glass wafers and
industrial ceramics based on proprietary technology, and also began a research program to develop
digital imaging technologies for niche industrial markets. The Companys products serve the screen
printing, awards and recognition, signage, electronics, aerospace, and industrial ceramics and


In [193]:
# backup
df_1 = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='first')
df_1.set_index('item', inplace=True)
raw_1 = document[doc][df_1['start'].loc['item1']:df_1['start'].loc['item2']]

df_2 = test_df[(test_df['item'] == 'item1') & (test_df['next_1'] == 'item2')]
df_2['difference'] = test_df['next_1_start'] - test_df['start']
df_2.sort_values('difference', ascending = False, inplace = True)
raw_2 = document[doc][df_2.iloc[0]['start']:df_2.iloc[0]['next_1_start']]

if (len(raw_1)>=len(raw_2)):
    item_1_raw = raw_1
else:
    item_1_raw = raw_2

item_1_content = BeautifulSoup(item_1_raw, 'lxml')
print(item_1_content.get_text("\n")[0:3500])

>Item 1.     
Business
 
 
 
 
This Annual Report on Form 10-K and
the documents incorporated herein by reference contain forward-looking statements based on expectations, estimates and projections as of the date of this filing. Actual results may differ materially from those expressed in forward-looking
statements. See Managements Discussion and Analysis of Financial Condition and Results of OperationsForward-Looking Statements. 
 
 
 
General 
 
 
 
Amazon.com, Inc. commenced operations on the World Wide Web in July 1995 and seeks to offer
Earths Biggest Selection. We seek to be the worlds most customer-centric company, where customers can find and discover anything they may want to buy online. We and our sellers list millions of unique items in categories such as books,
music, DVDs, videos, electronics, computers, camera and photo items, software, computer and video games, cell phones and service, tools and hardware, outdoor living items, kitchen and houseware products, toys, 

In [ ]:
# testing cells for new files that has 1a, 1b

In [15]:
#pos_dat.set_index('item', inplace=True)
item_1_raw = document['10-K'][6010:86306]
item_1_content = BeautifulSoup(item_1_raw, 'lxml')
print(item_1_content.get_text("\n")[0:3000])

ITEM 1. BUSINESS

     This Annual Report on Form 10-K and the documents incorporated herein by
reference contain forward-looking statements based on current expectations,
estimates and projections. Actual results may differ materially from those
expressed in forward-looking statements. See "Management's Discussion and
Analysis of Financial Condition and Results of Operations -- Forward-Looking
Statements."

GENERAL

     Amazon.com, Inc. is the world's leading online retailer. We have served
over 17 million customer accounts in over 150 countries. We directly offer for
sale millions of distinct items in categories such as books, music, DVDs,
videos, toys, electronics, software, video games and home improvement products.
Through our marketplace services such as Amazon.com Auctions, zShops and
sothebys.amazon.com, we have created Web-based marketplaces where buyers and
sellers can enter into transactions with respect to a wide range of products. In
addition to our US Web site, we curren